Inference, segmentation and synapse detection locally
==================================
This tutorial goes through steps to run inference and segmentation and synapse detection jobs using seuron deployed locally with docker compose. You should only use this notebook in jupyterlab hosted by seuron, and in order to run the examples, you need a linux system with a recent NVidia GPU with **CUDA 11.8+**. Make sure you follow the steps in the README file to create the local deployment.

Load seuronbot extension
========================
_The seuronbot extension can only be loaded by one notebook, trying to load another instance of the extension will result in an error. If you want to use the extension in a different notebook, make sure you reset the kernel of the current notebook to unload the extension_

In [ ]:
%load_ext seuronbot_ext

Visualize a small cutout in Minnie65
=====================================
In order to demonstrate the pipeline, we use a small cutout from Minnie65, which is the image stack FlyWire is based on. The bounding box of the cutout is [129285, 95090, 21304, 129797, 95602, 21496] at 8nm x 8nm x 40nm voxel size. The bbox is all we need to run the pipeline. In the cell below, The cutout are visualized using neuroglancer.

In [ ]:
from IPython.display import IFrame, HTML
import json
import urllib

ng_payload = {
  "dimensions": {
    "x": [8, "nm"],
    "y": [8, "nm"],
    "z": [40, "nm"],
  },
  "position": [129445, 95290, 21354],
  "layers": [
    {
      "type": "image",
      "source": "precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em",
      "tab": "rendering",
      "name": "minnie65"
    },
    {
      "type": "annotation",
      "source": {"url": "local://annotations"},
      "tool": "annotateBoundingBox",
      "tab": "annotations",
      "annotations": [
        {
          "pointA": [129285, 95090, 21304],
          "pointB": [129797, 95602, 21496],
          "type": "axis_aligned_bounding_box",
          "id": "6c88d2d28baae4c93a4d26736c039a8fc3c7d1ca"
        }
      ],
      "name": "annotation"
    }
  ],
  "showAxisLines": False,
  "showSlices": False,
  "layout": "xy-3d"
}
ng_link = f"https://neuroglancer-demo.appspot.com/#!{urllib.parse.quote(json.dumps(ng_payload))}"
display(IFrame(src=ng_link, width=800, height=600))

Run inference to create affinity map
====================================
To segment the cutout we picked, we first need to generate affinity map for it. The procedure is divided into two steps: First one submits the parameters describing the tasks, using the `update parameters` command. The bot performs sanity checks to prevent common mistakes. After the sanity check succeeds. One can launch the pipeline with `run pipeline` commands.

Submit inference parameters
--------------------------------------------------------
The command will send the python code in the current cell to seuronbot. The cell content must be **self-contained**, no reference to variables or modules from other cells. The cell have to define a `submit_parameter` function requiring no arguments. The function should return a dictionary containing the inference parameters or a list of dictionaries for several tasks. The bot will execute this function in a separate environment and use the returned parameters for sanity checks.

Seuron uses [the zettaai fork of chunkflow](https://github.com/ZettaAI/chunkflow) for inference tasks, `ranlu/chunkflow:zettaai` pointing to the latest build from the zettaiai. The parameters for each run can be divided into roughly two types: parameters for IO and parameters for the inference model. The IO parameters contain information of the input images: the location, resolution and bounding box, the output will be stored in `OUTPUT_PATH`, or `{OUTPUT_PREFIX}{NAME}` when `OUTPUT_PATH` is not defined. 
The parameters for the inference model must be adjusted for each model. You can find the parameters for the models we used below from [DeepEM](https://github.com/seung-lab/DeepEM/releases/tag/minnie) and [modelzoo](https://github.com/seung-lab/modelzoo/tree/main/models/IARPAphase2)

For local processes, we store the final and intermediate results to `/tmp`, which is automatically mounted from the host for all containers managed by seuron. If you want to use some other volume, make sure you add them to the deployment file and have proper permission.

In [ ]:
%%seuronbot update parameters
def submit_inference_parameters():
    bbox = [129285, 95090, 21304, 129797, 95602, 21496]
    common_param = {
        "IMAGE_PATH": "https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em",
        "IMAGE_FILL_MISSING": True,
        "IMAGE_RESOLUTION": [8, 8, 40],
        "BBOX": bbox,
        "OUTPUT_PREFIX": "file:///tmp/scratch/ng/test_aff/",
        "INHERIT_PARAMETERS": False,
        "CHUNKFLOW_IMAGE": "ranlu/chunkflow:trt",
        "MAX_RAM": 1,
    }
    aff_param = {
        "INPUT_PATCH_SIZE": [256,256,20],
        "OUTPUT_PATCH_SIZE": [192,192,16],
        "INFERENCE_OUTPUT_CHANNELS": 4,
        "OUTPUT_CHANNELS": 3,
        "MYELIN_MASK_THRESHOLD": 0.3,
        "INPUT_PATCH_OVERLAP_RATIO": 0.5,
        "ONNX_MODEL_PATH": "https://github.com/seung-lab/DeepEM/releases/download/minnie/mye-crop.onnx",
    }
    sem_param = {
        "INPUT_PATCH_SIZE": [256,256,20],
        "INFERENCE_OUTPUT_CHANNELS": 5,
        "OUTPUT_CHANNELS": 1,
        "OUTPUT_DTYPE": "uint8",
        "OUTPUT_LAYER_TYPE": "segmentation",
        "POSTPROC": "channel-voting",
        "INPUT_PATCH_OVERLAP_RATIO": 0.5,
        "ONNX_MODEL_PATH": "https://github.com/seung-lab/modelzoo/raw/main/models/IARPAphase2/SemanticSegmentation/model.onnx"
    }
    syn_param = {
        "INPUT_PATCH_SIZE": [192,192,18],
        "INFERENCE_OUTPUT_CHANNELS": 1,
        "INPUT_PATCH_OVERLAP_RATIO": 0.2,
        "ONNX_MODEL_PATH": "https://github.com/seung-lab/modelzoo/raw/main/models/IARPAphase2/SynapseDetection/model.onnx"
    }
    return [
                {
                    "NAME": "aff_local_test",
                    **common_param,
                    **aff_param,
                },
                {
                    "NAME": "sem_local_test",
                    **common_param,
                    **sem_param,
                },
                {
                    "NAME": "syn_local_test",
                    **common_param,
                    **syn_param,
                },
           ]

def submit_parameters():
    inf_param = submit_inference_parameters()
    return inf_param

Launch inference run
--------------------
Once the sanity check succeeded, we can start the inference using `%seuronbot run pipeline` command. The first message after the run is triggered is the cancel token to use if you want to cancel the run.

In [ ]:
%seuronbot run pipeline

Segment the affinity map
========================
After the affinity map is generated, we use it to create segmentation of the Flywire cutout. The procedure is similar to the inference tasks, first use `update parameter` command to submit the parameters describing the tasks, then use `run pipeline` to trigger the run.

Submit segmentation parameters
----------------------------------
For segmentation tasks seuron uses [abiss](https://github.com/seung-lab/abiss), `ranlu/abiss:main` is built from the main branch. The command creates both flat segmentations and inputs for [PyChunkedGraph](https://github.com/seung-lab/PyChunkedGraph). The segmentation requires affinity map as an input, make sure you update the path to the affinity map if you modified it in the previous step.

In [ ]:
%%seuronbot update parameters

def submit_segmentation_parameters(aff_path, sem_path):
    io_param = {
        "SCRATCH_PREFIX": "file:///tmp/scratch/",
        "IMAGE_PATH": "https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em",
        "AFF_PATH": aff_path,
        "SEM_PATH": sem_path,
        "NG_PREFIX": "file:///tmp/scratch/ng/",
    }
    seg_param = {
        "WS_HIGH_THRESHOLD": "0.99",
        "WS_LOW_THRESHOLD": "0.01",
        "WS_SIZE_THRESHOLD": "200",
        "AGG_THRESHOLD": "0.27",
        "WORKER_IMAGE": "ranlu/abiss:main",
        "SKIP_SKELETON": True,
    }
    return [
                {
                    "NAME": f"seg_local_test",
                    **io_param,
                    **seg_param,
                },
           ]

def submit_parameters():
    aff_path = "file:///tmp/scratch/ng/test_aff/aff_local_test"
    sem_path = "file:///tmp/scratch/ng/test_aff/sem_local_test"
    run_param = submit_segmentation_parameters(aff_path, sem_path)
    return run_param

Launch segmentation run
-----------------------
Same as inference, we can start the segmentation using `%seuronbot run pipeline` command. And you will receive the cancel token at the starting of the run

In [ ]:
%seuronbot run pipeline

Synapse Detection
=================
With the segmentation and voxel level synapses prediction, we use [Synaptor](https://github.com/ZettaAI/Synaptor) to detect synapses in the test cutout. The workflow is similar to inference and segmentation: submit the parameters using the `update parameters` then start the run with `run pipeline`

In [ ]:
%%seuronbot update parameters
def submit_synaptor_parameters():
    synaptor_param = {
        "Dimensions": {
          "blockshape": "64, 64, 64",
          "chunkshape": "256, 256, 64",
          "patchshape": "80, 80, 18",
          "startcoord": "129285, 95090, 21304",
          "volshape": "512, 512, 192",
          "voxelres": "8, 8, 40"
        },
        "Parameters": {
          "ccthresh": 0.27,
          "dustthresh": 0,
          "mergethresh": 0,
          "nummergetasks": 1,
          "szthresh": 40
        },
        "Provenance": {
          "motivation": "Testing postsynaptic terminal predictions"
        },
        "Volumes": {
          "baseseg": "file:///tmp/scratch/ng/seg/seg_local_test",
          "descriptor": "file:///tmp/scratch/ng/test_aff/syn_local_test",
          "image": "https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em",
          "output": "file:///tmp/scratch/ng/seg/syn_local_test",
          "tempoutput": "file:///tmp/scratch/ng/seg/syn_tmp"
        },
        "Workflow": {
          "connectionstr": "None",
          "maxclustersize": 1,
          "modelpath": "https://github.com/seung-lab/modelzoo/raw/main/models/IARPAphase2/SynapseAssignment/model.onnx",
          "queuename": "SHOULD_BE_SET_BY_AIRFLOW",
          "queueurl": "SHOULD_BE_SET_BY_AIRFLOW",
          "storagedir": "/tmp/scratch/ng/seg/syn_output/",
          "synaptor_image": "ranlu/synaptor:onnx",
          "workflowtype": "Segmentation+Assignment",
          "workspacetype": "File"
        }
    }
    return synaptor_param

def submit_parameters():
    inf_param = submit_synaptor_parameters()
    return inf_param

In [ ]:
%seuronbot run pipeline